In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from urllib.parse import quote
import time
from selenium.webdriver.chrome.options import Options
import requests
import json
import numpy as np
import pandas as pd
import tkinter as tk
import webbrowser

In [2]:
def productsearch(main):
    global pchome
    global s
    
    str(main)
    pchome = "https://ecshweb.pchome.com.tw/search/v3.3/?q="
    s = quote(main)
    ################################################################
    global sp1  
    global sp3
    global sp5
    
    sp1 = "https://shopee.tw/api/v2/search_items/?by=sales&keyword=" 
    sp3 = "&limit=50&newest="
    sp5 = "&order=desc&page_type=search&version=2"
    ##################################################################
    global yahoo
    global a
    global c

    yahoo = "https://tw.search.mall.yahoo.com/search/mall/product?p="
    a = "&pg="
    c = "&sort=popular"
    ##################################################################
    global momosearch
    mo = "https://www.momoshop.com.tw/search/searchShop.jsp?keyword="
    momosearch = mo + s

In [3]:
def pchomeinfo():
    global urlpc
    global len_urlpc
 
    options = webdriver.ChromeOptions()
    
    driver = webdriver.Chrome(executable_path='C:/Users/User/Downloads/chromedriver')
    driver.implicitly_wait(3)
    pchomesearch = pchome + s
    driver.get(pchomesearch)
    
    soup = BeautifulSoup(driver.page_source)
    for i in range(1, 5):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)

    namelist = []
    pricelist = []
    hreflist = []
    totallist = []
    
    for block in soup.select(".prod_name a"):
        name = block.text
        namelist.append(name)
    for block in soup.select(".price span"):
        price = int(block.text)
        pricelist.append(price)
    for link in driver.find_elements_by_xpath("//h5/*[@href]"):
        href = link.get_attribute('href')
        hreflist.append(href)
   
    for i in range(len(namelist)):
        total = []
        total.append(namelist[i])
        total.append(pricelist[i])
        total.append(hreflist[i])
        totallist.append(total)

    if pricemin != "no":
        output = []
        for i in range(len(totallist)):
            if totallist[i][1] > pricemin:
                output.append(totallist[i])
        pricetotal = []
        for i in range(len(output)):
            pricetotal.append(output[i][1])
            mini = pricetotal.index(min(pricetotal))
    else:
        output = totallist
        mini = 0
    
    urlpc = output[mini][2]
    len_urlpc = str(len(urlpc))

    return output[mini][2] + "\n" + output[mini][0] + "\n" + str(output[mini][1]) + "\n" 


In [4]:
def spinfo():
    global urlsp
    global len_urlsp
    
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36',
              "referer": "https://shopee.tw/search?keyword=%E9%9B%BB%E8%85%A6",
    }
     
    namelist = []
    pricelist = []
    hreflist = []
    totallist = []
    
    for i in range(5):
        sp4 = str(50*(i))
        res = requests.get(sp1+s+sp3+sp4+sp5, headers=headers)
        jd = res.json()
        for i in range(len(jd['items'])):
            name = jd['items'][i]['name']
            
            price = int(jd['items'][i]['price']/100000)
            
            sp_id = jd['items'][i]['shopid']
            m = "-i."
            item_id = jd['items'][i]['itemid']
            url = "https://shopee.tw/" + jd['items'][i]['name'] + m + str(sp_id) + "." + str(item_id)
            url = url.replace(" ", "-")
            
            namelist.append(name)
            pricelist.append(price)
            hreflist.append(url)
             
        for i in range(len(namelist)):
            total = []
            total.append(namelist[i])
            total.append(pricelist[i])
            total.append(hreflist[i])
            totallist.append(total)
            
    if pricemin != "no":
        output = []
        for i in range(len(totallist)):
            if totallist[i][1] > pricemin:
                output.append(totallist[i])
        pricetotal = []
        for i in range(len(output)):
            pricetotal.append(output[i][1])
            mini = pricetotal.index(min(pricetotal))
    else:
        output = totallist
        mini = 0
    
    urlsp = output[mini][2]
    len_urlsp = str(len(urlsp))

    return output[mini][2] + "\n" + output[mini][0] + "\n" + str(output[mini][1]) + "\n" 


In [5]:
def yahooinfo():
    global urlyahoo
    global len_urlyahoo
   
    totallist=[]
    
    for i in range(1, 5):
        A = []
        B = []
        C = []
       
        yahoosearch = yahoo + s + a + str(i) + c
        sp = BeautifulSoup(requests.get(yahoosearch).text, 'lxml')

        for i in sp.find_all(class_='BaseGridItem__title___2HWui'):
            values = list(i.stripped_strings)
            A.append(values)
        for i in sp.find_all(class_ = "BaseGridItem__price___31jkj"):
            values = (list(i.stripped_strings))[0]
            price = int(values.replace(",", "").replace("$", ""))
            B.append(price)
        for i in sp.find_all('a', class_="BaseGridItem__content___3LORP"):
            href = i.get('href')
            C.append(href)
        for i in range(len(A)):
            total = []
            total.append(A[i][0])
            total.append(B[i])
            total.append(C[i])
            totallist.append(total)
            
    if pricemin != "no":
        output = []
        for i in range(len(totallist)):
            if totallist[i][1] > pricemin:
                output.append(totallist[i])
        pricetotal = []
        for i in range(len(output)):
            pricetotal.append(output[i][1])
            mini = pricetotal.index(min(pricetotal))
    else:
        output = totallist
        mini = 0
        
    urlyahoo = output[mini][2]
    len_urlyahoo = str(len(urlyahoo))
    return output[mini][2] + "\n" + output[mini][0] + "\n" + str(output[mini][1]) + "\n" 
    

In [6]:
def momoinfo():
    global momosearch
    global urlmomo
    global len_urlmomo

    
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    driver = webdriver.Chrome('C:/Users/User/Downloads/chromedriver', options=options)
    name = []
    price = []
    link = []
    for i in range(1,5):
        driver.get(momosearch+"&searchType=6&curPage={}&_isFuzzy=0&showType=chessboardType".format(i))
        for i in range(len(driver.find_elements_by_class_name("prdName"))):
            name.append(driver.find_elements_by_class_name("prdName")[i].text)
        for i in range(len(driver.find_elements_by_class_name("price > b"))):
            price.append(driver.find_elements_by_class_name("price > b")[i].text.replace(",",""))
        for i in range(len(driver.find_elements_by_class_name("goodsUrl"))):
            z = (driver.find_elements_by_class_name("goodsUrl")[i]).get_attribute("href")
            link.append(z)
        time.sleep(3)

    price = np.array(price,dtype="float")
    dic = {"name":name,"price":price,"link":link}
    df = pd.DataFrame(dic)
    if pricemin != "no" :
        df1 = df[df["price"] > float(pricemin)]
        df2 = df1.sort_values(by="price")
        urlmomo = df2.iloc[0,2]
        len_urlmomo = str(len(urlmomo))
        return df2.iloc[0,2] + "\n" + df2.iloc[0,0] + "\n" + str(int(df2.iloc[0,1])) + "\n"
    else :
        urlmomo = df.iloc[0,2]
        len_urlmomo = str(len(urlmomo))
        return df.iloc[0,2] + "\n" + df.iloc[0,0] + "\n" + str(int(df.iloc[0,1])) + "\n"


In [7]:
def show_arrow_cursor(event):
    t1.config(cursor = "arrow")
    t2.config(cursor = "arrow")
    t3.config(cursor = "arrow")
    t4.config(cursor = "arrow")

def show_xterm_cursor(event):
    t1.config(cursor = "xterm")
    t2.config(cursor = "xterm")
    t3.config(cursor = "xterm")
    t4.config(cursor = "xterm")

def click1(event):
    webbrowser.open(urlyahoo)
def click2(event):
    webbrowser.open(urlpc)
def click3(event):
    webbrowser.open(urlsp)
def click4(event):
    webbrowser.open(urlmomo)

In [8]:
def web_crawler():
    global pricemin
    main = e.get()
    if price_input.get()!="":
        pricemin = int(price_input.get())
    else:
        pricemin = "no"
    productsearch(main)
    
    t1.delete('1.0','end')
    t1.insert('insert', yahooinfo())
    t1.tag_add("link", "1.0", "1."+len_urlyahoo)
    t1.tag_config("link", foreground = "blue", underline = True)
    t1.tag_bind("link", "<Enter>", show_arrow_cursor)
    t1.tag_bind("link", "<Leave>", show_xterm_cursor)
    t1.tag_bind("link", "<Button-1>", click1)

    t2.delete('1.0','end')
    t2.insert('insert', pchomeinfo())
    t2.tag_add("link", "1.0", "1."+len_urlpc)
    t2.tag_config("link", foreground = "blue", underline = True)
    t2.tag_bind("link", "<Enter>", show_arrow_cursor)
    t2.tag_bind("link", "<Leave>", show_xterm_cursor)
    t2.tag_bind("link", "<Button-1>", click2)
    
    t3.delete('1.0','end')
    t3.insert('insert', spinfo())
    t3.tag_add("link", "1.0", "1."+len_urlsp)
    t3.tag_config("link", foreground = "blue", underline = True)
    t3.tag_bind("link", "<Enter>", show_arrow_cursor)
    t3.tag_bind("link", "<Leave>", show_xterm_cursor)
    t3.tag_bind("link", "<Button-1>", click3)
    
    t4.delete('1.0','end')
    t4.insert('insert', momoinfo())
    t4.tag_add("link", "1.0", "1."+len_urlmomo)
    t4.tag_config("link", foreground = "blue", underline = True)
    t4.tag_bind("link", "<Enter>", show_arrow_cursor)
    t4.tag_bind("link", "<Leave>", show_xterm_cursor)
    t4.tag_bind("link", "<Button-1>", click4)

In [9]:
window=tk.Tk()
window.title('ShopMoya')
window.geometry('600x800')

var = tk.StringVar() 
var.set('A')

def print_selection1():
    l2.config(text=''+var.get())
    price_input.delete(0, 'end')
    price_input.pack_forget()
def print_selection2():
    l2.config(text=''+var.get())
    price_input.pack()

l1 = tk.Label(window,text='想買什麼哩？',bg='pink',font=('微軟正黑體',12),width=15,height=1)
e = tk.Entry(window,show=None)
rd1 = tk.Radiobutton(window,text='依熱門度',variable=var,value='最熱門！',command=print_selection1)
rd2 = tk.Radiobutton(window,text='依價格',variable=var,value='請輸入最低價格',command=print_selection2)
l2 = tk.Label(window,bg='pink',font=('微軟正黑體',12),width=15,height=1,text='尚未選擇')


price_input = tk.Entry(window,show=None)
submit_button = tk.Button(window,text='Submit',width=15,height=1,command=web_crawler)

t1 = tk.Text(window,bg='orange',width=200,height=10)
t2 = tk.Text(window,bg='white',width=200,height=10)
t3 = tk.Text(window,bg='orange',width=200,height=10)
t4 = tk.Text(window,bg='white',width=200,height=10)

s1 = tk.Label(window,text='yahoo!超級商城',bg='FloralWhite',font=('微軟正黑體',12),width=155,height=1,anchor="nw")
s2 = tk.Label(window,text='PChome 24h購物',bg='FloralWhite',font=('微軟正黑體',12),width=155,height=1,anchor="nw")
s3 = tk.Label(window,text='蝦皮購物',bg='FloralWhite',font=('微軟正黑體',12),width=155,height=1,anchor="nw")
s4 = tk.Label(window,text='momo購物網',bg='FloralWhite',font=('微軟正黑體',12),width=155,height=1,anchor="nw")

l1.pack()
e.pack()
rd1.pack()
rd2.pack()
l2.pack()

submit_button.place(x=240,y=180)

s1.place(x=0,y=210)
t1.place(x=0,y=235)
s2.place(x=0,y=350)
t2.place(x=0,y=375)
s3.place(x=0,y=490)
t3.place(x=0,y=515)
s4.place(x=0,y=630)
t4.place(x=0,y=655)
window.mainloop()